In [1]:
from rdflib import Graph, Namespace, Literal, URIRef
from rdflib.namespace import RDF, RDFS, OWL, XSD

In [2]:
g = Graph()
g.parse("social_kg.ttl", format="turtle")

<Graph identifier=N02ae080ae560429a8886174f31cea273 (<class 'rdflib.graph.Graph'>)>

In [3]:
ont_iri = g.value(None, RDF.type, OWL.Ontology)
print("\nOntology IRI:", ont_iri)

ONTO = Namespace(ont_iri + "#")
g.bind("onto", ONTO)

DATA = Namespace(ont_iri + "/data/")
g.bind("data", DATA)


Ontology IRI: http://www.semanticweb.org/kaushalamancherla/ontologies/2025/4/social_KG_mini


In [4]:
print("XSD base URI:", str(XSD))
print("OWL base URI:", str(OWL))
print("RDF base URI:", str(RDF))
print("DATA base URI:", str(DATA))
print("ONTO base URI:", str(ONTO))

XSD base URI: http://www.w3.org/2001/XMLSchema#
OWL base URI: http://www.w3.org/2002/07/owl#
RDF base URI: http://www.w3.org/1999/02/22-rdf-syntax-ns#
DATA base URI: http://www.semanticweb.org/kaushalamancherla/ontologies/2025/4/social_KG_mini/data/
ONTO base URI: http://www.semanticweb.org/kaushalamancherla/ontologies/2025/4/social_KG_mini#


In [5]:
'''
Q1 

“For a given researcher, list all institutions they have been affiliated with, 
along with the start and end years of each affiliation.”
'''

'\nQ1 \n\n“For a given researcher, list all institutions they have been affiliated with, \nalong with the start and end years of each affiliation.”\n'

In [6]:
q1 = f"""
PREFIX onto: <{ONTO}>
PREFIX data: <{DATA}>
PREFIX xsd: <{XSD}>

SELECT ?institution ?startDate ?endDate WHERE {{
  <{DATA}person/A5065824713> onto:hasAssociation ?assoc .
  ?assoc onto:associatedInstitution ?institution ;
         onto:startDate ?startDate ;
         onto:endDate ?endDate .
}}
ORDER BY ?startDate
"""

In [7]:
print("=== Q1: Person -> Institution affiliations ===")
for row in g.query(q1):
    print(row)

=== Q1: Person -> Institution affiliations ===
(rdflib.term.URIRef('http://www.semanticweb.org/kaushalamancherla/ontologies/2025/4/social_KG_mini/data/institution/I1334627681'), rdflib.term.Literal('2017-01-01', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#date')), rdflib.term.Literal('2019-12-31', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#date')))
(rdflib.term.URIRef('http://www.semanticweb.org/kaushalamancherla/ontologies/2025/4/social_KG_mini/data/institution/I1334627681'), rdflib.term.Literal('2021-01-01', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#date')), rdflib.term.Literal('2024-12-31', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#date')))
(rdflib.term.URIRef('http://www.semanticweb.org/kaushalamancherla/ontologies/2025/4/social_KG_mini/data/institution/I1306266525'), rdflib.term.Literal('2022-01-01', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#date')), rdflib.term.Literal('2023-12-31', d

In [8]:
'''
Q2

“Given an institution A, find its parent institution(s) B (and any related‐to peers C).”
'''

'\nQ2\n\n“Given an institution A, find its parent institution(s) B (and any related‐to peers C).”\n'

In [9]:
q2 = f"""
PREFIX onto: <{ONTO}>
PREFIX data: <{DATA}>

SELECT ?parentInst ?relatedInst WHERE {{
  OPTIONAL {{ <{DATA}institution/I1334627681> onto:hasParentInstitution ?parentInst }}
  OPTIONAL {{ <{DATA}institution/I1334627681> onto:relatedInstitution   ?relatedInst }}
}}
"""

In [10]:
print("=== Q2: Parent & Related Institutions ===")
for row in g.query(q2):
    print(row)

=== Q2: Parent & Related Institutions ===
(rdflib.term.URIRef('http://www.semanticweb.org/kaushalamancherla/ontologies/2025/4/social_KG_mini/data/institution/I122411786'), None)
(rdflib.term.URIRef('http://www.semanticweb.org/kaushalamancherla/ontologies/2025/4/social_KG_mini/data/institution/I4210124779'), None)


In [11]:
'''
Q3

“List all researchers who have collaborated with a given institution X 
(i.e., Person A worked on a project to which Institution X also contributed).”
'''

'\nQ3\n\n“List all researchers who have collaborated with a given institution X \n(i.e., Person A worked on a project to which Institution X also contributed).”\n'

In [12]:
q3 = f"""
PREFIX onto: <{ONTO}>
PREFIX data: <{DATA}>

SELECT DISTINCT ?person
WHERE {{
  <{DATA}institution/I2799356940>  onto:worksOn  ?proj .
  ?person  onto:worksOn  ?proj ;
           a            onto:Person .
  FILTER (?person != <{DATA}organization/I2799356940>)
}}
"""

In [13]:
print("=== Q3: Researchers who collaborated with a given institution ===")
for row in g.query(q3):
    print(row)

=== Q3: Researchers who collaborated with a given institution ===
(rdflib.term.URIRef('http://www.semanticweb.org/kaushalamancherla/ontologies/2025/4/social_KG_mini/data/person/A5065824713'),)
(rdflib.term.URIRef('http://www.semanticweb.org/kaushalamancherla/ontologies/2025/4/social_KG_mini/data/person/A5047816335'),)
(rdflib.term.URIRef('http://www.semanticweb.org/kaushalamancherla/ontologies/2025/4/social_KG_mini/data/person/A5087062396'),)


In [14]:
'''
Q4

“For institution I, list each topic T it has been associated with, 
along with the project completion dates when that association first appeared (and last appeared, if many).”
'''

'\nQ4\n\n“For institution I, list each topic T it has been associated with, \nalong with the project completion dates when that association first appeared (and last appeared, if many).”\n'

In [15]:
q4 = f"""
PREFIX onto: <{ONTO}>
PREFIX data: <{DATA}>

SELECT ?topic
       (MIN(?projDate) AS ?firstDate)
       (MAX(?projDate) AS ?lastDate)
WHERE {{
  <{DATA}institution/I1334627681>  onto:worksOn           ?proj .
  ?conf  onto:linkSourceProject  ?proj ;
         onto:linkTargetTopic    ?topic ;
         onto:confidenceScore    ?score .
  ?proj  onto:completedDate     ?projDate .
}}
GROUP BY ?topic
ORDER BY ?firstDate
"""

In [16]:
print("=== Q4: Institution & Topic Timelines ===")
for row in g.query(q4):
    print(row)

=== Q4: Institution & Topic Timelines ===
(rdflib.term.URIRef('http://www.semanticweb.org/kaushalamancherla/ontologies/2025/4/social_KG_mini/data/topic/T10251'), rdflib.term.Literal('1992-09-11T00:00:00', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#dateTime')), rdflib.term.Literal('2024-07-01T00:00:00', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#dateTime')))
(rdflib.term.URIRef('http://www.semanticweb.org/kaushalamancherla/ontologies/2025/4/social_KG_mini/data/topic/T10325'), rdflib.term.Literal('1992-09-11T00:00:00', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#dateTime')), rdflib.term.Literal('2025-05-16T00:00:00', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#dateTime')))
(rdflib.term.URIRef('http://www.semanticweb.org/kaushalamancherla/ontologies/2025/4/social_KG_mini/data/topic/T11786'), rdflib.term.Literal('1992-09-11T00:00:00', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#dateTime')), rdflib.t

In [17]:
'''
Q5

“For researcher R, what are all the topics they have published on, and in which years did those first/last occur?”
'''

'\nQ5\n\n“For researcher R, what are all the topics they have published on, and in which years did those first/last occur?”\n'

In [18]:
q5 = f"""
PREFIX onto: <{ONTO}>
PREFIX data: <{DATA}>

SELECT ?topic (MIN(?pubDate) AS ?firstYear) (MAX(?pubDate) AS ?lastYear) WHERE {{
  <{DATA}person/A5065824713> onto:worksOn ?proj .
  ?proj onto:resultsIn ?paper .
  ?conf onto:linkSourceProject ?proj ;
        onto:linkTargetTopic ?topic .
  ?paper onto:producedDate ?pubDate .
}}
GROUP BY ?topic
ORDER BY ?firstYear
"""

In [19]:
print("\n=== Q5: Researcher-Topic timelines ===")
for row in g.query(q5):
    print(row)


=== Q5: Researcher-Topic timelines ===
(rdflib.term.URIRef('http://www.semanticweb.org/kaushalamancherla/ontologies/2025/4/social_KG_mini/data/topic/T10159'), rdflib.term.Literal('2014-04-01T00:00:00', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#dateTime')), rdflib.term.Literal('2024-05-01T00:00:00', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#dateTime')))
(rdflib.term.URIRef('http://www.semanticweb.org/kaushalamancherla/ontologies/2025/4/social_KG_mini/data/topic/T10251'), rdflib.term.Literal('2014-04-01T00:00:00', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#dateTime')), rdflib.term.Literal('2022-10-19T00:00:00', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#dateTime')))
(rdflib.term.URIRef('http://www.semanticweb.org/kaushalamancherla/ontologies/2025/4/social_KG_mini/data/topic/T11786'), rdflib.term.Literal('2014-04-01T00:00:00', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#dateTime')), rdflib.ter

In [20]:
'''
Q6

“Find all pairs of researchers (Person A, Person B) who co-produced at least one project, 
and report the project ID and its completion date.”

'''

'\nQ6\n\n“Find all pairs of researchers (Person A, Person B) who co-produced at least one project, \nand report the project ID and its completion date.”\n\n'

In [21]:
q6 = f"""
PREFIX onto: <{ONTO}>
PREFIX data: <{DATA}>
PREFIX xsd: <{XSD}>

SELECT DISTINCT ?pA ?pB ?proj ?projDate
WHERE {{
  ?pA  a            onto:Person ;  onto:worksOn ?proj .
  ?pB  a            onto:Person ;  onto:worksOn ?proj .
  FILTER (STR(?pA) < STR(?pB))          # ← compare lexicographically as strings
  ?proj onto:completedDate ?projDate .
}}
ORDER BY ?pA ?pB ?projDate
"""

print("=== Q6: Person pairs co-working on same project ===")
for row in g.query(q6):
    print(row)

=== Q6: Person pairs co-working on same project ===
(rdflib.term.URIRef('http://www.semanticweb.org/kaushalamancherla/ontologies/2025/4/social_KG_mini/data/person/A5005240098'), rdflib.term.URIRef('http://www.semanticweb.org/kaushalamancherla/ontologies/2025/4/social_KG_mini/data/person/A5020685599'), rdflib.term.URIRef('http://www.semanticweb.org/kaushalamancherla/ontologies/2025/4/social_KG_mini/data/project/W2962201654'), rdflib.term.Literal('2019-07-01T00:00:00', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#dateTime')))
(rdflib.term.URIRef('http://www.semanticweb.org/kaushalamancherla/ontologies/2025/4/social_KG_mini/data/person/A5005240098'), rdflib.term.URIRef('http://www.semanticweb.org/kaushalamancherla/ontologies/2025/4/social_KG_mini/data/person/A5020685599'), rdflib.term.URIRef('http://www.semanticweb.org/kaushalamancherla/ontologies/2025/4/social_KG_mini/data/project/W2993768087'), rdflib.term.Literal('2019-12-04T00:00:00', datatype=rdflib.term.URIRef('http

In [22]:
'''
Q7

“Find all pairs of institutions (Inst A, Inst B) who co-produced or co-funded at least one project, 
and report the project ID and its completion date.”

'''

'\nQ7\n\n“Find all pairs of institutions (Inst A, Inst B) who co-produced or co-funded at least one project, \nand report the project ID and its completion date.”\n\n'

In [23]:
q7_inst_pairs = f"""
PREFIX onto: <{ONTO}>
PREFIX data: <{DATA}>
PREFIX xsd:  <{XSD}>

SELECT DISTINCT ?instA ?instB ?proj ?projDate
WHERE {{
  {{
    # (a)  co-produce via worksOn
    ?instA a onto:Institution ; onto:worksOn  ?proj .
    ?instB a onto:Institution ; onto:worksOn  ?proj .
  }}
  UNION
  {{
    # (b)  co-funding via hasFunded
    ?instA a onto:Institution ; onto:hasFunded ?proj .
    ?instB a onto:Institution ; onto:hasFunded ?proj .
  }}

  # ---- 2.  Keep each unordered pair only once ----
  FILTER(STR(?instA) < STR(?instB))

  # ---- 3.  Pull a date for the project ----
  OPTIONAL {{ ?proj onto:completedDate ?dCompleted }}
  OPTIONAL {{ ?proj onto:startingDate  ?dStarted   }}
  BIND(COALESCE(?dCompleted, ?dStarted) AS ?projDate)
}}
ORDER BY ?instA ?instB ?projDate
LIMIT 100
"""

In [24]:
print("=== Q7: Institution pairs that co-fund / co-produce the same project ===")
for row in g.query(q7_inst_pairs):
    print(row)

=== Q7: Institution pairs that co-fund / co-produce the same project ===
(rdflib.term.URIRef('http://www.semanticweb.org/kaushalamancherla/ontologies/2025/4/social_KG_mini/data/institution/I103635307'), rdflib.term.URIRef('http://www.semanticweb.org/kaushalamancherla/ontologies/2025/4/social_KG_mini/data/institution/I1334627681'), rdflib.term.URIRef('http://www.semanticweb.org/kaushalamancherla/ontologies/2025/4/social_KG_mini/data/project/W2026339618'), rdflib.term.Literal('2008-05-01T00:00:00', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#dateTime')))
(rdflib.term.URIRef('http://www.semanticweb.org/kaushalamancherla/ontologies/2025/4/social_KG_mini/data/institution/I103635307'), rdflib.term.URIRef('http://www.semanticweb.org/kaushalamancherla/ontologies/2025/4/social_KG_mini/data/institution/I22299242'), rdflib.term.URIRef('http://www.semanticweb.org/kaushalamancherla/ontologies/2025/4/social_KG_mini/data/project/W2026339618'), rdflib.term.Literal('2008-05-01T00:00:0

In [25]:
'''
Q8

“For a given paper P, list all papers its relatedTo and all its cited works, including the authors of the cited works.”

'''

'\nQ8\n\n“For a given paper P, list all papers its relatedTo and all its cited works, including the authors of the cited works.”\n\n'

In [26]:
paper_id   = "W2897473609"
paper_uri  = f"{DATA}paper/{paper_id}"

q8 = f"""
PREFIX onto: <{ONTO}>
PREFIX xsd:  <{XSD}>

SELECT ?relationType ?targetPaper ?paperTitle ?author
WHERE {{
  #  (A) “relatedToPaper” branch
  {{
     <{paper_uri}>  onto:relatedToPaper  ?targetPaper .
     BIND("relatedTo" AS ?relationType)
  }}
  UNION
  # ------------------------------------------------------------------
  #  (B) “referencesOrCites” branch
  # ------------------------------------------------------------------
  {{
     <{paper_uri}>  onto:referencesOrCites  ?targetPaper .
     BIND("cites" AS ?relationType)

     #   For cited papers, pull each author (Person) that produced it
     OPTIONAL {{ ?targetPaper onto:producedBy ?author }}
  }}

  # ------------------------------------------------------------------
  #  Common bits: human-readable display name for the target paper
  # ------------------------------------------------------------------
  OPTIONAL {{ ?targetPaper onto:name ?paperTitle }}
}}
ORDER BY ?relationType ?targetPaper ?author
"""

In [27]:
print("=== Q8: Related & Cited network for", paper_uri, "===")
for row in g.query(q8):
    print(row)

=== Q8: Related & Cited network for http://www.semanticweb.org/kaushalamancherla/ontologies/2025/4/social_KG_mini/data/paper/W2897473609 ===
(rdflib.term.Literal('cites'), rdflib.term.URIRef('http://www.semanticweb.org/kaushalamancherla/ontologies/2025/4/social_KG_mini/data/paper/W2012232666'), rdflib.term.Literal('Space Weather and the Global Positioning System'), rdflib.term.URIRef('http://www.semanticweb.org/kaushalamancherla/ontologies/2025/4/social_KG_mini/data/person/A5076361258'))
(rdflib.term.Literal('cites'), rdflib.term.URIRef('http://www.semanticweb.org/kaushalamancherla/ontologies/2025/4/social_KG_mini/data/paper/W2735156630'), rdflib.term.Literal('Finding multiscale connectivity in our geospace observational system: Network analysis of total electron content'), rdflib.term.URIRef('http://www.semanticweb.org/kaushalamancherla/ontologies/2025/4/social_KG_mini/data/institution/I1334627681'))
(rdflib.term.Literal('cites'), rdflib.term.URIRef('http://www.semanticweb.org/kaushal

In [28]:
'''
Q9

“Identify all authors (Person A) whose papers cite the output of a particular researcher B’s papers.”

'''

'\nQ9\n\n“Identify all authors (Person A) whose papers cite the output of a particular researcher B’s papers.”\n\n'

In [29]:
B_ID = "A5065824713"
person_B = f"{DATA}person/{B_ID}"

q9 = f"""
PREFIX onto: <{ONTO}>
PREFIX xsd:  <{XSD}>

SELECT DISTINCT ?authorA ?citingPaper ?citingDate ?citedPaper
WHERE {{
  <{person_B}>   onto:producesOutput   ?citedPaper .

  ?citingPaper   onto:referencesOrCites   ?citedPaper .
  ?citingPaper   onto:producedBy          ?authorA .
  ?authorA       a                        onto:Person .

  OPTIONAL {{ ?citingPaper onto:producedDate ?citingDate }}

  FILTER(?authorA != <{person_B}>)
}}
ORDER BY ?authorA ?citingDate ?citingPaper ?citedPaper
"""

In [30]:
print("=== Q9: Authors citing researchers ===")
for row in g.query(q9):
    print(row)

=== Q9: Authors citing researchers ===
(rdflib.term.URIRef('http://www.semanticweb.org/kaushalamancherla/ontologies/2025/4/social_KG_mini/data/person/A5025952050'), rdflib.term.URIRef('http://www.semanticweb.org/kaushalamancherla/ontologies/2025/4/social_KG_mini/data/paper/W4399020461'), rdflib.term.Literal('2024-05-01T00:00:00', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#dateTime')), rdflib.term.URIRef('http://www.semanticweb.org/kaushalamancherla/ontologies/2025/4/social_KG_mini/data/paper/W2735156630'))
(rdflib.term.URIRef('http://www.semanticweb.org/kaushalamancherla/ontologies/2025/4/social_KG_mini/data/person/A5047816335'), rdflib.term.URIRef('http://www.semanticweb.org/kaushalamancherla/ontologies/2025/4/social_KG_mini/data/paper/W2735156630'), rdflib.term.Literal('2017-07-01T00:00:00', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#dateTime')), rdflib.term.URIRef('http://www.semanticweb.org/kaushalamancherla/ontologies/2025/4/social_KG_mini/dat

In [31]:
'''
Q10

“For a given topic T0, list all sibling or related topics T1, T2, …, with their names.”

'''

'\nQ10\n\n“For a given topic T0, list all sibling or related topics T1, T2, …, with their names.”\n\n'

In [32]:
# ------------------------------------------------------------------
# 1)  Pick the seed topic T0
# ------------------------------------------------------------------
T0_ID     = "T10159"                              # ← replace with the local/OpenAlex ID you want
topic_T0  = f"{DATA}topic/{T0_ID}"                # full IRI of the seed topic

# ------------------------------------------------------------------
# 2)  SPARQL query: find all related topics and their names
# ------------------------------------------------------------------
q10 = f"""
PREFIX onto: <{ONTO}>

SELECT DISTINCT ?related ?name
WHERE {{
  # -- in case you stored the edge in only one direction -----------
  {{
     <{topic_T0}>  onto:relatedToTopic  ?related .
  }}
  UNION
  {{
     ?related      onto:relatedToTopic  <{topic_T0}> .
  }}

  # human-readable label (if stored during ingestion)
  OPTIONAL {{ ?related  onto:displayName  ?name }}
}}
ORDER BY ?related
"""

print("=== Topics related to ===")
for row in g.query(q10):
    print(row)

=== Topics related to ===
(rdflib.term.URIRef('http://www.semanticweb.org/kaushalamancherla/ontologies/2025/4/social_KG_mini/data/topic/T10026'), rdflib.term.Literal('Galaxies: Formation, Evolution, Phenomena'))
(rdflib.term.URIRef('http://www.semanticweb.org/kaushalamancherla/ontologies/2025/4/social_KG_mini/data/topic/T10039'), rdflib.term.Literal('Stellar, planetary, and galactic studies'))
(rdflib.term.URIRef('http://www.semanticweb.org/kaushalamancherla/ontologies/2025/4/social_KG_mini/data/topic/T10095'), rdflib.term.Literal('Cosmology and Gravitation Theories'))
(rdflib.term.URIRef('http://www.semanticweb.org/kaushalamancherla/ontologies/2025/4/social_KG_mini/data/topic/T10251'), rdflib.term.Literal('Solar and Space Plasma Dynamics'))
(rdflib.term.URIRef('http://www.semanticweb.org/kaushalamancherla/ontologies/2025/4/social_KG_mini/data/topic/T10325'), rdflib.term.Literal('Astro and Planetary Science'))
(rdflib.term.URIRef('http://www.semanticweb.org/kaushalamancherla/ontologies

In [33]:
'''
Q11

“For a given institution I and topic T, list every project that I funded which maps to T, 
including the topic-confidence score. Separately, show how that support has changed over time 
(e.g., number of funded projects and average confidence per year).”

'''

'\nQ11\n\n“For a given institution I and topic T, list every project that I funded which maps to T, \nincluding the topic-confidence score. Separately, show how that support has changed over time \n(e.g., number of funded projects and average confidence per year).”\n\n'

In [34]:
# ╔═══════════════════════ 1. PICK INSTITUTION & TOPIC ═════════════════════╗
INST_ID  = "I4210124779"          # ← institution’s OpenAlex/local ID (NASA)
TOPIC_ID = "T10159"               # ← topic’s OpenAlex/local ID

inst_uri  = f"{DATA}institution/{INST_ID}"
topic_uri = f"{DATA}topic/{TOPIC_ID}"

# ╠═══════════════════════ 2. RAW LIST OF FUNDED PROJECTS ══════════════════╣
q_funded_raw = f"""
PREFIX onto: <{ONTO}>
PREFIX xsd:  <{XSD}>

SELECT ?proj ?projDate ?score
WHERE {{
  <{inst_uri}>  onto:hasFunded  ?proj .

  ?conf  onto:linkSourceProject  ?proj ;
         onto:linkTargetTopic    <{topic_uri}> ;
         onto:confidenceScore    ?score .

  OPTIONAL {{ ?proj onto:completedDate ?projDate }}
  OPTIONAL {{ ?proj onto:startingDate  ?projDate }}
}}
ORDER BY DESC(?score) ?projDate
"""

print(f"— RAW projects funded by {INST_ID} on topic {TOPIC_ID} —")
for row in g.query(q_funded_raw):
    print(row)

# ╠═══════════════════════ 3. YEAR-BY-YEAR TREND ═══════════════════════════╣
q_funded_trend = f"""
PREFIX onto: <{ONTO}>
PREFIX xsd:  <{XSD}>

SELECT ?year (COUNT(?proj) AS ?numProjects) (AVG(?score) AS ?avgScore)
WHERE {{
  <{inst_uri}>  onto:hasFunded  ?proj .

  ?conf  onto:linkSourceProject  ?proj ;
         onto:linkTargetTopic    <{topic_uri}> ;
         onto:confidenceScore    ?score .

  OPTIONAL {{ ?proj onto:completedDate ?d }}
  OPTIONAL {{ ?proj onto:startingDate  ?d }}
  FILTER(BOUND(?d))

  # ------------- extract calendar year --------------------
  BIND( YEAR(?d) AS ?year )
}}
GROUP BY ?year
ORDER BY ?year
"""

print(f"\n— TREND for {INST_ID} on topic {TOPIC_ID} —")
for row in g.query(q_funded_trend):
    print(row)


— RAW projects funded by I4210124779 on topic T10159 —
(rdflib.term.URIRef('http://www.semanticweb.org/kaushalamancherla/ontologies/2025/4/social_KG_mini/data/project/W2070202309'), rdflib.term.Literal('1995-01-01T00:00:00', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#dateTime')), rdflib.term.Literal('1.0', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#double')))
(rdflib.term.URIRef('http://www.semanticweb.org/kaushalamancherla/ontologies/2025/4/social_KG_mini/data/project/W1969396600'), rdflib.term.Literal('1996-11-01T00:00:00', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#dateTime')), rdflib.term.Literal('1.0', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#double')))
(rdflib.term.URIRef('http://www.semanticweb.org/kaushalamancherla/ontologies/2025/4/social_KG_mini/data/project/W1997574807'), rdflib.term.Literal('1997-09-15T00:00:00', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#dateTime')), rdflib.ter

In [35]:
'''
Q12

“For every connected researcher pair (A, B),
show the first year they became connected, the last year they appear connected,
and the total count of distinct years in which any connecting event happened.”

'''

'\nQ12\n\n“For every connected researcher pair (A, B),\nshow the first year they became connected, the last year they appear connected,\nand the total count of distinct years in which any connecting event happened.”\n\n'

In [36]:
'''
Q13

“For every pair of institutions (I₁, I₂) that share a topic T,
return each contiguous time-interval during which both were active on T, together with the exact start and end dates of
the overlap, the number of calendar-years it spans, and each institution’s own active span for T.”*
'''

'\nQ13\n\n“For every pair of institutions (I₁, I₂) that share a topic T,\nreturn each contiguous time-interval during which both were active on T, together with the exact start and end dates of\nthe overlap, the number of calendar-years it spans, and each institution’s own active span for T.”*\n'

In [37]:
q_topic_overlap = f"""
PREFIX onto: <{ONTO}>
PREFIX xsd:  <{XSD}>

SELECT ?inst1 ?inst2 ?topic
       ?overlapStart ?overlapEnd
       (xsd:integer(YEAR(?overlapEnd) - YEAR(?overlapStart) + 1) AS ?numYears)
       ?start1 ?end1 ?start2 ?end2
WHERE {{

  # — institution 1 episode on topic T —
  ?inst1 a onto:Institution ;
         onto:hasAssociation ?ep1 .
  ?ep1   onto:associatedTopic ?topic ;
         onto:startDate       ?start1 ;
         onto:endDate         ?end1 .

  # — institution 2 episode on the SAME topic —
  ?inst2 a onto:Institution ;
         onto:hasAssociation ?ep2 .
  ?ep2   onto:associatedTopic ?topic ;
         onto:startDate       ?start2 ;
         onto:endDate         ?end2 .

  FILTER(?inst1 != ?inst2 && STR(?inst1) < STR(?inst2))

  # --- intersection of the two intervals --------------------------
  BIND( IF(?start1 > ?start2, ?start1, ?start2) AS ?overlapStart )
  BIND( IF(?end1   < ?end2,   ?end1,   ?end2)   AS ?overlapEnd )
  FILTER(?overlapStart <= ?overlapEnd)

}}
GROUP BY ?inst1 ?inst2 ?topic
         ?overlapStart ?overlapEnd
         ?start1 ?end1 ?start2 ?end2
ORDER BY DESC(?numYears) ?topic ?inst1 ?inst2 ?overlapStart
"""

print("— Institutional topic-overlap windows —")
for row in g.query(q_topic_overlap):
    print(row)


— Institutional topic-overlap windows —
(rdflib.term.URIRef('http://www.semanticweb.org/kaushalamancherla/ontologies/2025/4/social_KG_mini/data/institution/I1334627681'), rdflib.term.URIRef('http://www.semanticweb.org/kaushalamancherla/ontologies/2025/4/social_KG_mini/data/institution/I82495205'), rdflib.term.URIRef('http://www.semanticweb.org/kaushalamancherla/ontologies/2025/4/social_KG_mini/data/topic/T10251'), rdflib.term.Literal('2009-01-01T00:00:00', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#dateTime')), rdflib.term.Literal('2014-12-31T23:59:00', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#dateTime')), rdflib.term.Literal('6', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#integer')), rdflib.term.Literal('2008-01-01T00:00:00', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#dateTime')), rdflib.term.Literal('2020-12-31T23:59:00', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#dateTime')), rdflib.term

In [38]:
'''
Q14

“For every pair of researchers (A = the citer, B = the cited),
list the first year A’s papers cited B’s work, the last year it happened,
and the total number of citing events.”

'''

'\nQ14\n\n“For every pair of researchers (A = the citer, B = the cited),\nlist the first year A’s papers cited B’s work, the last year it happened,\nand the total number of citing events.”\n\n'

In [39]:
q_citation_timeline = f"""
PREFIX onto: <{ONTO}>
PREFIX xsd:  <{XSD}>

SELECT ?citer ?cited
       (MIN(?yr) AS ?firstYear)
       (MAX(?yr) AS ?lastYear)
       (COUNT(?paperA) AS ?numCites)
WHERE {{
  # PaperA produced by CITER
  ?paperA  onto:producedBy         ?citer ;
           onto:referencesOrCites  ?paperB ;
           onto:producedDate       ?d .

  # PaperB produced by CITED
  ?paperB  onto:producedBy         ?cited .

  FILTER(?citer != ?cited)               # exclude self-citation

  # extract calendar year of the citing paper
  BIND( YEAR(?d) AS ?yr )
}}
GROUP BY ?citer ?cited
ORDER BY DESC(?numCites) ?citer ?cited
"""

print("— Citation timeline between researcher pairs —")
for row in g.query(q_citation_timeline):
    print(row)

— Citation timeline between researcher pairs —
(rdflib.term.URIRef('http://www.semanticweb.org/kaushalamancherla/ontologies/2025/4/social_KG_mini/data/institution/I1334627681'), rdflib.term.URIRef('http://www.semanticweb.org/kaushalamancherla/ontologies/2025/4/social_KG_mini/data/person/A5087062396'), rdflib.term.Literal('2004', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#integer')), rdflib.term.Literal('2024', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#integer')), rdflib.term.Literal('114', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#integer')))
(rdflib.term.URIRef('http://www.semanticweb.org/kaushalamancherla/ontologies/2025/4/social_KG_mini/data/institution/I1334627681'), rdflib.term.URIRef('http://www.semanticweb.org/kaushalamancherla/ontologies/2025/4/social_KG_mini/data/person/A5076361258'), rdflib.term.Literal('2004', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#integer')), rdflib.term.Literal('2025', dataty

In [40]:
'''
Q15

“For every pair of researchers (A, B) who overlapped at the
same institution I, find cases where A and B worked on at least one project during their
overlap.
'''

'\nQ15\n\n“For every pair of researchers (A, B) who overlapped at the\nsame institution I, find cases where A and B worked on at least one project during their\noverlap.\n'

In [41]:
# TODO